# **DATA PREPROCESSING BASED ON EDA INSIGHTS**



##  Home Loan Dataset: Preprocessing Overview

This notebook continues from the **Exploratory Data Analysis (EDA)** phase, where data quality checks, missing value imputation, and outlier treatment were completed.
We now focus on preparing the dataset for machine learning by engineering relevant features, transforming skewed variables, encoding categorical features, and scaling numerical ones.
All steps directly align with the evidence-based recommendations from the *Home Loan EDA Report*.

### **Preprocessing Plan (Post-EDA)**

1. **Feature Engineering**

   * Create derived variables to improve model interpretability and predictive power:

     * `Total_Income` = `ApplicantIncome` + `CoapplicantIncome`
     * `Loan_to_Income_Ratio` = `LoanAmount` / `Total_Income`
     * `Income_per_Dependent` = `ApplicantIncome` / (`Dependents`)
   * Generate interaction terms where relevant:

     * `Married × Total_Income`
     * `Education × Self_Employed`

2. **Skewness Handling**

   * Apply **log transformation** to right-skewed variables:

     * `ApplicantIncome`, `CoapplicantIncome`, and `LoanAmount`
   * Recheck skewness visually (histogram/QQ plot) to confirm normalization.

3. **Encoding Categorical Variables**

   * **Label Encoding** for binary features:

     * `Gender`, `Education`, `Self_Employed`, `Married`
   * **One-Hot Encoding** for nominal multi-class features:

     * `Property_Area`
   * **Ordinal Encoding** for ordered categories:

     * `Credit_History` (0 = poor, 1 = good)

4. **Feature Scaling**

   * Standardize continuous variables using `StandardScaler`:

     * `ApplicantIncome`, `CoapplicantIncome`, `LoanAmount`, `Total_Income`, `Loan_to_Income_Ratio`

5. **Feature Selection**

   * Retain only high-signal variables identified in EDA:

     * `Credit_History`, `Total_Income`, `LoanAmount`, `Education`, `Married`, `Property_Area`
   * Drop low-variance or redundant variables like `Loan_Amount_Term` (mostly 360 months).

6. **Target Variable Encoding**

   * Encode the binary target:

     * `Loan_Status`: map `Y → 1`, `N → 0`

7. **Train-Test Split**

   * Perform a **stratified 80/20 split** to maintain the same proportion of approved vs. rejected loans.

---

### **Key EDA Evidence Driving These Steps**

| EDA Finding                                  | Preprocessing Action                |
| -------------------------------------------- | ----------------------------------- |
| Income and loan amount highly skewed         | Apply log transformation            |
| `Credit_History` most predictive of approval | Retain as main predictor            |
| `Loan_Amount_Term` offers low variance       | Drop or treat as constant           |
| Property area affects loan status            | One-hot encode                      |
| Married and Total_Income interact            | Create interaction feature          |
| Dependents slightly affect approval odds     | Engineer income-per-dependent ratio |



#### **1. Import Libraries and Load Data**

In [27]:
# core libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# preprocessing libraries
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report, confusion_matrix

# Statistical libraries
from scipy import stats
from scipy.stats import zscore, skew

# Set style for better visualizations
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")

Libraries imported successfully!


In [28]:
# The URL of the raw train data CSV file
# url = 'https://raw.githubusercontent.com/ek-chris/Practice_datasets/refs/heads/main/home_loan_train.csv'

# Read the CSV file directly into a pandas DataFrame
df = pd.read_csv("cleaned_home_data.csv")
print("Dataset loaded successfully!")
print("\nFirst 5 rows of the DataFrame:")
df.head()

# Create a copy for preprocessing
df_processed = df.copy()

Dataset loaded successfully!

First 5 rows of the DataFrame:


In [29]:
df_processed.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,128.0,360.0,1.0,Urban,1
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,0
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,1
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,1
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,1


In [30]:
df_processed.columns

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [31]:
# setting load_id as the index
df_processed.set_index("Loan_ID", inplace = True)

#### **2. EDA-Based Data Quality Assessment**

In [32]:
# 1. Check for missing values (EDA showed no missing values)
print("1. Missing Values")
missing_values = df_processed.isnull().sum()

if missing_values.sum() > 0:
    print(missing_values[missing_values > 0])
else:
    print("No missing values found (as expected from EDA)")

# 2. Check for duplicates
print("\n2. Duplicate Rows:")
duplicates = df_processed.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")
if duplicates > 0:
    print(f"Percentage of duplicates: {(duplicates / len(df_processed)) * 100:.2f}%")
else:
    print("No duplicated values found (as expected from EDA)")

# 3. Check skewness for variables identified in EDA as right-skewed
print("\n3. Skewness Analysis (EDA identified right-skewed variables):")
skewed_vars = ["ApplicantIncome", "CoapplicantIncome", "LoanAmount"]
for var in skewed_vars:
    if var in df_processed.columns:
        skewness = skew(df_processed[var])
        print(f"{var}: skewness = {skewness:.3f} "
              f"({'right-skewed' if skewness > 0.5 else 'approximately normal'})")
    else:
        print(f"{var}: not found in dataset")

# 4. Check correlation with target (EDA evidence)
print("\n4. Correlation with Target Variable (Loan_Status):")
target_col = "Loan_Status"

if target_col in df_processed.columns:
    correlations = df_processed.corr(numeric_only=True)[target_col].sort_values(
        key=abs, ascending=False
    )

    print("High-signal features (|correlation| > 0.2):")
    high_signal = correlations[abs(correlations) > 0.2].drop(target_col, errors="ignore")
    for feature, corr in high_signal.items():
        print(f"  {feature}: {corr:.3f}")

    print("\nLow-signal features (|correlation| < 0.1):")
    low_signal = correlations[abs(correlations) < 0.1]
    for feature, corr in low_signal.items():
        print(f"  {feature}: {corr:.3f}")

else:
    print(f"Target column '{target_col}' not found in df_processed")

1. Missing Values
No missing values found (as expected from EDA)

2. Duplicate Rows:
Number of duplicate rows: 0
No duplicated values found (as expected from EDA)

3. Skewness Analysis (EDA identified right-skewed variables):
ApplicantIncome: skewness = 6.524 (right-skewed)
CoapplicantIncome: skewness = 7.473 (right-skewed)
LoanAmount: skewness = 2.736 (right-skewed)

4. Correlation with Target Variable (Loan_Status):
High-signal features (|correlation| > 0.2):
  Credit_History: 0.541

Low-signal features (|correlation| < 0.1):
  CoapplicantIncome: -0.059
  LoanAmount: -0.033
  Loan_Amount_Term: -0.023
  Dependents: 0.010
  ApplicantIncome: -0.005


In [33]:
df_processed.columns

Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

## Handle Missing Values

In [34]:
df_processed.isna().sum()

Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [35]:
# checking numerical columns
numerical = df_processed.select_dtypes(include=["int64", "float64"]).columns
numerical

Index(['Dependents', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Loan_Status'],
      dtype='object')

In [36]:
df_processed.isna().sum()

Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

#### **3. Handle Duplicates**


In [37]:
# Remove duplicates if any (EDA didn't report duplicates, but let's be thorough)
if duplicates > 0:
    print(f"Removing {duplicates} duplicate rows...")
    df_processed = df_processed.drop_duplicates()
    print(f"Dataset shape after removing duplicates: {df_processed.shape}")
else:
    print("No duplicates to remove (as expected from EDA)")


No duplicates to remove (as expected from EDA)


#### **5. Outlier Treatment (EDA Recommendation)**

In [38]:
# print("6. OUTLIER DETECTION AND TREATMENT (IQR-CAPPING METHOD)")
# print("EDA revealed presence of outliers in ApplicantIncome, CoapplicantIncome, and LoanAmount.")

# # Variables confirmed in EDA to have outliers
# outlier_features = ["ApplicantIncome", "CoapplicantIncome", "LoanAmount"]

# outliers_capped = 0

# for col in outlier_features:
#     if col in df_processed.columns:
#         Q1 = df_processed[col].quantile(0.25)
#         Q3 = df_processed[col].quantile(0.75)
#         IQR = Q3 - Q1
#         lower_bound = Q1 - 1.5 * IQR
#         upper_bound = Q3 + 1.5 * IQR
        
#         # Count outliers before capping
#         outliers_before = ((df_processed[col] < lower_bound) | (df_processed[col] > upper_bound)).sum()
        
#         if outliers_before > 0:
#             # Cap outliers
#             df_processed[col] = np.where(df_processed[col] < lower_bound, lower_bound, df_processed[col])
#             df_processed[col] = np.where(df_processed[col] > upper_bound, upper_bound, df_processed[col])
#             outliers_capped += outliers_before
#             print(f"{col}: Capped {outliers_before} outliers")
#         else:
#             print(f"{col}: No outliers detected")
#     else:
#         print(f"{col}: not found in dataset — skipped")

# print(f"\nTotal outliers capped: {outliers_capped}")
# print(f"Dataset shape after outlier treatment: {df_processed.shape}")
# print("Outlier treatment complete. Dataset ready for feature engineering.\n")

#### **6. Feature Engineering**

Implement the specific feature engineering recommendations from the EDA report

**Total Income**

In [39]:
print("FEATURE ENGINEERING BASED ON EDA RECOMMENDATIONS")
print("Creating new derived variables to improve model interpretability and performance.\n")

# 1. Total_Income: Combine ApplicantIncome and CoapplicantIncome
if all(col in df_processed.columns for col in ["ApplicantIncome", "CoapplicantIncome"]):
    df_processed["Total_Income"] = df_processed["ApplicantIncome"] + df_processed["CoapplicantIncome"]
    print("Created 'Total_Income' (Applicant + Coapplicant)")
else:
    print("Could not create 'Total_Income'; missing one or more income columns")
#2. Loan_to_Income= Total_Income/LoanAmount
print(f"Dataset shape after feature engineering: {df_processed.shape}")
print("Feature engineering complete. Ready for encoding and scaling.\n")


FEATURE ENGINEERING BASED ON EDA RECOMMENDATIONS
Creating new derived variables to improve model interpretability and performance.

Created 'Total_Income' (Applicant + Coapplicant)
Dataset shape after feature engineering: (614, 13)
Feature engineering complete. Ready for encoding and scaling.



In [40]:
df_processed.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Total_Income
Loan_ID,,,,,,,,,,,,,
LP001002,Male,No,0,Graduate,No,5849,0.0,128.0,360.0,1.0,Urban,1,5849.0
LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,0,6091.0
LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,1,3000.0
LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,1,4941.0
LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,1,6000.0


In [41]:
# coverting loan amount term datatype from float to int
df_processed["Loan_Amount_Term"]=df_processed["Loan_Amount_Term"].astype(int)
df_processed["Loan_Amount_Term"]

Loan_ID
LP001002    360
LP001003    360
LP001005    360
LP001006    360
LP001008    360
           ... 
LP002978    360
LP002979    180
LP002983    360
LP002984    360
LP002990    360
Name: Loan_Amount_Term, Length: 614, dtype: int64

In [42]:
# putting loan amountterm from years to month
df_processed["Loan_Amount_Term_Year"]= round((df_processed["Loan_Amount_Term"]/12),1 ).astype(int)
df_processed["Loan_Amount_Term_Year"].head()

Loan_ID
LP001002    30
LP001003    30
LP001005    30
LP001006    30
LP001008    30
Name: Loan_Amount_Term_Year, dtype: int64

**EMI: LoanAmount/Loan_Amount_Term_Year**

In [43]:
# 3. EMI: LoanAmount/Loan_Amount_Term_Year
if all(col in df_processed.columns for col in ["LoanAmount", "Loan_Amount_Term_Year"]):
    df_processed["EMI"] = df_processed["LoanAmount"] / df_processed["Loan_Amount_Term_Year"]
    print("Created 'EMI': LoanAmount + Loan_Amount_Term_Year)")
else:
    print("Could not create 'Total_Income'; missing one or more income columns")
    
df_processed["EMI"]    

Created 'EMI': LoanAmount + Loan_Amount_Term_Year)


Loan_ID
LP001002    4.266667
LP001003    4.266667
LP001005    2.200000
LP001006    4.000000
LP001008    4.700000
              ...   
LP002978    2.366667
LP002979    2.666667
LP002983    8.433333
LP002984    6.233333
LP002990    4.433333
Name: EMI, Length: 614, dtype: float64

**Has CoApplicant**

In [44]:
df_processed.Married.unique()

array(['No', 'Yes'], dtype=object)

| Numeric | Category   | Meaning               |
| ------- | ---------- | --------------------- |
| 1       | **Yes**    | Married               |
| 0       | **No**     | Not Married           |


In [ ]:
# mapping the marital status
df_processed["Marital_status"] = df_processed["Married"].map({"Yes": 1, "No": 0})    

**Dependents**

In [ ]:
# checking the datatypes of the dependents columns
df_processed["Dependents"].dtypes

In [51]:
df_processed["Dependents"]=df_processed["Dependents"].astype(int)

In [52]:
df_processed.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Total_Income,Loan_Amount_Term_Year,EMI,Marital_status,ApplicantIncome_log,CoapplicantIncome_log,LoanAmount_log
Loan_ID,,,,,,,,,,,,,,,,,,,
LP001002,Male,No,0,Graduate,No,5849,0.0,128.0,360,good,Urban,NaN,5849.0,30,4.266667,0,8.674026,0.000000,4.852030
LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360,good,Rural,NaN,6091.0,30,4.266667,1,8.430109,7.319202,4.852030
LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360,good,Urban,NaN,3000.0,30,2.200000,1,8.006368,0.000000,4.189655
LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360,good,Urban,NaN,4941.0,30,4.000000,1,7.856707,7.765993,4.787492
LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360,good,Urban,NaN,6000.0,30,4.700000,0,8.699515,0.000000,4.948760


**Income Category**

| Numeric | Category   | Meaning               |
| ------- | ---------- | --------------------- |
| <= 1500 | **Low**    | Low Income            |
| > 1500  | **Medium** | Medium Income         |
| > 5000  | **High**   | High Income           |


In [55]:
def map_income_category(income):
    if income <= 1500:
        return "Low"
    elif (income > 1500) and (income <= 5000):
        return "Medium"
    elif income > 5000:
        return "High"
    else:
        return "Very High"
    
df_processed["Income_category"] = df_processed["Total_Income"].apply(map_income_category)
df_processed["Income_category"]    

Loan_ID
LP001002      High
LP001003      High
LP001005    Medium
LP001006    Medium
LP001008      High
             ...  
LP002978    Medium
LP002979    Medium
LP002983      High
LP002984      High
LP002990    Medium
Name: Income_category, Length: 614, dtype: object

#### **4. Log-Transform Skewed Variables (EDA Recommendation)**

Based on EDA findings, transform the right-skewed variables identified

In [47]:
# Log-transform skewed variables as recommended by EDA
print("LOG-TRANSFORMING SKEWED VARIABLES")
print("EDA identified these variables as right-skewed and recommended log transformation:")

# Variables to log-transform based on EDA findings
def handled_skewed_var(df):
    for var in skewed_vars:
        if var in df_processed.columns:
            # checking if variable has zero or negative values
            min_val = df_processed[var].min()
            if min_val <= 0:
                # use log1p for variales with zeros
                df_processed[f'{var}_log'] = np.log1p(df_processed[var])
                print(f"{var}: Applied log1p transformation (had{min_val:.3f} minimum value)")
            else:
                # use log for positive values only
                df_processed[f'{var}_log'] = np.log(df_processed[var])
                print(f"{var}: APplied log transformation")
                
                # check skewness befire and after
                original_skew = skew(df_processed[var])
                transformed_skew = skew(df_processed[f'{var}_log'])
                
                print(f"Original Skewness: {original_skew:.3f}, Transformed Skewness: {transformed_skew:.3f}")
    return df
print(f"\nDataset shape after log transformation: {df_processed.shape}")
print("New log-transformes columns: ", [col for col in df_processed.columns if "_log" in col])

LOG-TRANSFORMING SKEWED VARIABLES
EDA identified these variables as right-skewed and recommended log transformation:

Dataset shape after log transformation: (614, 16)
New log-transformes columns:  []


In [48]:
handled_skewed_var(df_processed)

ApplicantIncome: APplied log transformation
Original Skewness: 6.524, Transformed Skewness: 0.478
CoapplicantIncome: Applied log1p transformation (had0.000 minimum value)
LoanAmount: APplied log transformation
Original Skewness: 2.736, Transformed Skewness: -0.195


,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Total_Income,Loan_Amount_Term_Year,EMI,Marital_status,ApplicantIncome_log,CoapplicantIncome_log,LoanAmount_log
Loan_ID,,,,,,,,,,,,,,,,,,,
LP001002,Male,No,0,Graduate,No,5849,0.0,128.0,360,1.0,Urban,1,5849.0,30,4.266667,0,8.674026,0.000000,4.852030
LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360,1.0,Rural,0,6091.0,30,4.266667,1,8.430109,7.319202,4.852030
LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360,1.0,Urban,1,3000.0,30,2.200000,1,8.006368,0.000000,4.189655
LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360,1.0,Urban,1,4941.0,30,4.000000,1,7.856707,7.765993,4.787492
LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360,1.0,Urban,1,6000.0,30,4.700000,0,8.699515,0.000000,4.948760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360,1.0,Rural,1,2900.0,30,2.366667,0,7.972466,0.000000,4.262680
LP002979,Male,Yes,3,Graduate,No,4106,0.0,40.0,180,1.0,Rural,1,4106.0,15,2.666667,1,8.320205,0.000000,3.688879
LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360,1.0,Urban,1,8312.0,30,8.433333,1,8.996157,5.484797,5.533389


#### **6. Encoding**

In [49]:
# mapping loan_status from abject to int to check correlation 
df_processed["Loan_Status"] = df_processed["Loan_Status"].map({"Y":1,"N": 0})

In [50]:
# mapping credit history from int to object to check correlation
df_processed["Credit_History"] = df_processed["Credit_History"].map({1.0:"good", 0.0:"bad"})